# Pydantic as a Toolkit to Define Schemas

Basics of Pydantic to write down abstract schemas that can be used to validate data.

In [1]:
!pip install pydantic

/bin/bash: pip: command not found


In [2]:
from pydantic import BaseModel

ModuleNotFoundError: No module named 'pydantic'

In [ ]:
class Dogs(BaseModel):
    breed: str
    toy: str

# An instance
Bella = Dogs(breed = "Poodle", toy="ball")
print(Bella)

In [20]:
from typing import Literal
from enum import Enum

class Hair(Enum):
    Curly = "curly"
    Straight = "straight"
    Shaved = "shaved"

class Dogs(BaseModel):
    breed: str
    toy: str
    hair: Hair
    color: Literal["brown", "white"]

In [ ]:
# Charlie is not valid because of toy and color
Charlie = Dogs(breed="Corgi", toy=12, hair=Hair.Straight, color="red")

In [26]:
from pydantic import Field, field_validator, model_validator
from typing import Optional

class Dogs(BaseModel):
    breed: str = Field(description="The name of an existing breed")
    toy: str = Field(max_length=50)
    hair: Hair = Field(description="The hair type of the dog")
    color: Optional[Literal["brown", "white"]] = Field(default=None)
    age: int = Field(default=1, gt=0, description="Age must be greater than 0")

    @field_validator('breed') 
    def check_name(cls, value):
        if any(char in value for char in "!@#^*"):
            raise ValueError('Breed cannot contain special characters')
        return value
    
    # Adding a new cross-field validator to the Dogs class
    @model_validator(mode='before')
    def check_hair_and_color(cls, values):
        hair = values.get('hair')
        color = values.get('color')
        # If hair is shaved, color can be None or absent
        if hair == Hair.Shaved and color is not None:
            raise ValueError("Dogs with shaved hair should not have a color.")
        # If hair is curly or straight, color must be specified and cannot be None
        if hair in {Hair.Curly, Hair.Straight} and color is None:
            raise ValueError("Dogs with curly or straight hair must have a color.")
        return values

In [ ]:
# Coco is not valid because of the hair-color inconsistency
Coco = Dogs(breed = "Beagle", toy = "socks", hair = Hair.Curly, age = 1)

In [ ]:
pydantic_to_json = Dogs.model_json_schema()
print(pydantic_to_json)